In [10]:
# Weights and Biases related imports
import wandb
from wandb.keras import WandbMetricsLogger

In [11]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='bytes')
    return batch

def load_cifar10_data(folder_path):
    train_data = []
    train_labels = []

    for i in range(1, 6):
        batch_file = f"{folder_path}/data_batch_{i}"
        batch = load_cifar10_batch(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    test_batch_file = f"{folder_path}/test_batch"
    test_batch = load_cifar10_batch(test_batch_file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']

    train_data = np.vstack(train_data)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

def preprocess_data(train_data, train_labels, test_data, test_labels):
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    train_labels_onehot = to_categorical(train_labels)
    test_labels_onehot = to_categorical(test_labels)

    return train_data, train_labels_onehot, test_data, test_labels_onehot

cifar10_folder = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(cifar10_folder)

x_train, y_train, x_test, y_test = preprocess_data(
    train_data, train_labels, test_data, test_labels
)

print("Train Data Shape:", x_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Data Shape:", x_test.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (50000, 32, 32, 3)
Train Labels Shape: (50000, 10)
Test Data Shape: (10000, 32, 32, 3)
Test Labels Shape: (10000, 10)


In [12]:
# os.environ['WANDB_NOTEBOOK_NAME'] = 'RUN_1'
wandb.login()

True

In [13]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'learning_rate': {
        'values': np.logspace(-6, -3, num=6).tolist()
        },
    'batch_size': {
          'values': [64, 128]
        },
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'earlystopping_patience': {
        'value': 10},
    'epochs': {
        'value': 100}
    })

In [14]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'values': [64, 128]},
                'earlystopping_patience': {'value': 10},
                'epochs': {'value': 100},
                'learning_rate': {'values': [1e-06,
                                             3.981071705534969e-06,
                                             1.584893192461114e-05,
                                             6.309573444801929e-05,
                                             0.00025118864315095795,
                                             0.001]}}}


In [15]:
sweep_id = wandb.sweep(sweep_config, project="CIFAR-10_Classification")

Create sweep with ID: bd946eq7
Sweep URL: https://wandb.ai/takim/CIFAR-10_Classification/sweeps/bd946eq7


In [16]:
import tensorflow as tf

# Define the AlexNet architecture
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(2, 2), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1)),

        tf.keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1)),

        tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
        tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
        tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1)),
        
        tf.keras.layers.Flatten(),
        
        tf.keras.layers.Dense(units=4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(units=4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(units=10, activation='softmax')
    ])
    return model

In [17]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def train(config = None):
    with wandb.init(config=config):

        config = wandb.config

        tf.keras.backend.clear_session()
        model = create_model()
        model.compile(
            optimizer = Adam(learning_rate=config["learning_rate"]),
            loss = "categorical_crossentropy",
            metrics = ["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top@3_accuracy')]
        )

        early_stopping = EarlyStopping(monitor='val_loss',
                                    patience=config["earlystopping_patience"],
                                    restore_best_weights=True)

        model.fit(x_train, y_train,
                                    epochs=config["epochs"],
                                    batch_size=config["batch_size"],
                                    validation_split=0.1,
                                    callbacks=[
                                        WandbMetricsLogger(log_freq='epoch'),
                                        early_stopping
                                    ], verbose=1
                                    )

In [18]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: cuq1w728 with config:
wandb: 	batch_size: 64
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 1e-06
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 19s 22ms/step - loss: 4.5696 - accuracy: 0.1417 - top@3_accuracy: 0.3779 - val_loss: 1.9470 - val_accuracy: 0.2982 - val_top@3_accuracy: 0.6214
Epoch 2/100
704/704 [==============================] - 15s 22ms/step - loss: 2.3122 - accuracy: 0.2194 - top@3_accuracy: 0.5167 - val_loss: 1.8280 - val_accuracy: 0.3460 - val_top@3_accuracy: 0.6816
Epoch 3/100
704/704 [==============================] - 15s 22ms/step - loss: 2.0419 - accuracy: 0.2749 - top@3_accuracy: 0.5941 - val_loss: 1.7546 - val_accuracy: 0.3662 - val_top@3_accuracy: 0.7082
Epoch 4/100
704/704 [==============================] - 15s 22ms/step - loss: 1.9136 - accuracy: 0.3069 - top@3_accuracy: 0.6432 - val_loss: 1.6924 - val_accuracy: 0.3906 - val_top@3_accuracy: 0.7320
Epoch 5/100
704/704 [==============================] - 15s 22ms/step - loss: 1.8220 - accuracy: 0.3398 - top@3_accuracy: 0.6777 - val_loss: 1.6386 - val_accuracy: 0.4052 - val_top@3_accuracy: 0.7456
Epoch

epoch/accuracy,▁▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████
epoch/val_accuracy,▁▂▃▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████
epoch/val_loss,█▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_top@3_accuracy,▁▃▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
epoch/accuracy,0.744
epoch/epoch,84
epoch/learning_rate,0.0


wandb: Agent Starting Run: lc5747im with config:
wandb: 	batch_size: 64
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 3.981071705534969e-06
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 15s 20ms/step - loss: 3.3022 - accuracy: 0.2020 - top@3_accuracy: 0.4808 - val_loss: 1.7869 - val_accuracy: 0.3576 - val_top@3_accuracy: 0.6972
Epoch 2/100
704/704 [==============================] - 14s 20ms/step - loss: 1.8970 - accuracy: 0.3190 - top@3_accuracy: 0.6508 - val_loss: 1.6667 - val_accuracy: 0.3974 - val_top@3_accuracy: 0.7432
Epoch 3/100
704/704 [==============================] - 14s 20ms/step - loss: 1.7387 - accuracy: 0.3727 - top@3_accuracy: 0.7116 - val_loss: 1.5662 - val_accuracy: 0.4366 - val_top@3_accuracy: 0.7734
Epoch 4/100
704/704 [==============================] - 14s 20ms/step - loss: 1.6340 - accuracy: 0.4120 - top@3_accuracy: 0.7470 - val_loss: 1.5061 - val_accuracy: 0.4528 - val_top@3_accuracy: 0.7866
Epoch 5/100
704/704 [==============================] - 14s 20ms/step - loss: 1.5544 - accuracy: 0.4432 - top@3_accuracy: 0.7704 - val_loss: 1.4318 - val_accuracy: 0.4852 - val_top@3_accuracy: 0.8082
Epoch

...

epoch/accuracy,▁▃▃▄▄▅▅▅▅▆▆▆▇▇▇▇████
epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁
epoch/top@3_accuracy,▁▄▄▅▅▆▆▆▆▇▇▇▇▇██████
epoch/val_accuracy,▁▃▄▅▆▇▇▇███▇▇██▇████
epoch/val_loss,█▅▅▃▃▂▁▁▁▁▂▂▂▃▃▄▄▅▆▆
epoch/val_top@3_accuracy,▁▅▄▆▇▇█▇▇█▇▇▇████▇▇█
epoch/accuracy,0.89478
epoch/epoch,19
epoch/learning_rate,0.00025


wandb: Agent Starting Run: 2q49gq7g with config:
wandb: 	batch_size: 128
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 0.001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
352/352 [==============================] - 13s 35ms/step - loss: 3.3208 - accuracy: 0.0992 - top@3_accuracy: 0.2996 - val_loss: 2.3029 - val_accuracy: 0.0958 - val_top@3_accuracy: 0.2878
Epoch 2/100
352/352 [==============================] - 12s 34ms/step - loss: 2.3030 - accuracy: 0.0970 - top@3_accuracy: 0.2957 - val_loss: 2.3030 - val_accuracy: 0.0976 - val_top@3_accuracy: 0.2964
Epoch 3/100
352/352 [==============================] - 12s 34ms/step - loss: 2.3029 - accuracy: 0.0984 - top@3_accuracy: 0.2984 - val_loss: 2.3026 - val_accuracy: 0.0976 - val_top@3_accuracy: 0.2990
Epoch 4/100
352/352 [==============================] - 11s 32ms/step - loss: 2.3029 - accuracy: 0.0986 - top@3_accuracy: 0.2971 - val_loss: 2.3026 - val_accuracy: 0.1058 - val_top@3_accuracy: 0.3072
Epoch 5/100
352/352 [==============================] - 12s 33ms/step - loss: 2.3030 - accuracy: 0.0995 - top@3_accuracy: 0.3008 - val_loss: 2.3028 - val_accuracy: 0.0976 - val_top@3_accuracy: 0.2984
Epoch

epoch/accuracy,▆▂▄▅▆▅█▇▇▆▇▇▆▂▆▄▃▇▁▇
epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▆▂▅▄▇▇█▄▆▆▇▆▆▄▄▄▂▄▁▆
epoch/val_accuracy,▂▃▃█▃▇▂▃▂█▇▃▂▃▂▃▃▁▂▂
epoch/val_loss,▄▄▂▂▃▃█▄▃▁▃▅▄▂▃▃▃▃▃▃
epoch/val_top@3_accuracy,▁▄▅█▅▅▂▃▅▇▄▂▁█▁▂▁▄▁▁
epoch/accuracy,0.1
epoch/epoch,19
epoch/learning_rate,0.001


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
